## Load data

In [1]:
import sys
from config import dataset_dir
sys.path.append('{}/lovit_textmining_dataset'.format(dataset_dir))

from navernews_10days import get_news_paths
from soynlp.utils import DoublespaceLineCorpus

corpus_path = get_news_paths(date='2016-10-20')
corpus = DoublespaceLineCorpus(corpus_path, iter_sent=True)

soynlp=0.0.49
Dataset version
[navermovie_comments.data] is latest (0.0.1)
[navermovie_comments.models] is latest (0.0.1)
[navernews_10days.data] is latest (0.0.1)
[navernews_10days.models] is latest (0.0.1)


## L-R graph

L-R graph는 L 왼쪽에 어떤 R들이 등장하는지 확인하기 위한 그래프로 dict[L]에는 [R]들이 몇 번 오른쪽에 등장하였는지의 빈도수 입니다. 

In [2]:
from collections import defaultdict
lr_graph = defaultdict(lambda: defaultdict(int))

for sent in corpus:
    for eojeol in sent.split():
        for e in range(1, len(eojeol) + 1):
            (l, r) = (eojeol[:e], eojeol[e:])
            lr_graph[l][r] += 1

L인 word가 주어지면 R들의 분포를 가져오기 위해서 get 함수를 이용합시다. 

In [3]:
def get_r(word, topk=-1):
    r = sorted(lr_graph.get(word, {}).items(), key=lambda x:-x[1])
    if topk > 0:
        return r[:topk]
    return r

word = '드라마'
for r, freq in get_r(word, 20):
    print('%s - %s: %d' % (word, r, freq))

드라마 - : 1268
드라마 - 를: 164
드라마 - 다: 152
드라마 - 의: 140
드라마 - 로: 138
드라마 - 에서: 98
드라마 - 와: 62
드라마 - 에: 55
드라마 - 는: 55
드라마 - 가: 48
드라마 - 이다: 24
드라마 - 라는: 15
드라마 - 인: 14
드라마 - 하우스: 14
드라마 - 나: 13
드라마 - 관계자는: 8
드라마 - 도: 8
드라마 - 틱했던: 7
드라마 - 에서는: 6
드라마 - 타운: 5


## R feature score table

세종말뭉치에 존재하는 r set 들의 명사 가능 점수를 계산한 table 을 로딩합니다. 탭으로 구분된 텍스트 파일입니다.

```
을까를	   -1.000000
스러워하는	    1.000000
다주는	   -1.000000
냔	    0.080570
하다보니	    1.000000
...
```

In [4]:
with open('noun_score_sejong', encoding='utf-8') as f:
    r_score = {}
    for line in f:
        r, score = line.strip().split('\t')
        score = float(score)
        r_score[r] = score
    print('num r features = %d' % len(r_score))

num r features = 2398


## Noun classification with r fetaures

단어 word 가 주어졌을 때, r features 를 가져온 뒤, 이를 이용하여 명사 가능 점수를 계산해 봅시다.

r_features 를 가져온 뒤, 각 r 의 명사 가능 점수를, 빈도수를 weight 로 하여 가중평균 합니다.

In [6]:
def predict(word):
    score = 0.0
    norm = 0
    for r, freq in get_r(word):
        if (r in r_score) == False:
            continue
        score += r_score.get(r) * freq
        norm += freq

    if norm != 0:
        score /= norm

    return score


for word in ['박근', '박근혜', '대통령', '정', '정부', '정부의', '알아', '알아냈']:
    print('score({}) = {:.3}'.format(word, predict(word)))

score(박근) = -0.968
score(박근혜) = 0.478
score(대통령) = 0.492
score(정) = 0.323
score(정부) = 0.574
score(정부의) = 0.0
score(알아) = -0.784
score(알아냈) = -0.885


## Limitation & Solutions

하지만 실제 명사 추출은 이보다 복잡한 prediction 과정 및 후처리가 필요합니다.

한 예로, `아이디`라는 단어는 실제로는 이날의 뉴스에서 명사로 자주 이용되지만, `아이디어`라는 명사의 `-어` 때문에 명사가 아닌 것처럼 판단될 수 있습니다. `되어`, `먹어` 처럼 `-어`는 대표적인 어미이기 때문입니다. 이를 해결할 수 있는 간단한 방법으로는 길이가 긴 subword 에 대하여 먼저 명사 점수를 계산한 뒤, 그 subword 가 명사이면 L-R graph 에서 이에 해당하는 모든 조합을 지우는 것입니다. 

이 방법을 이용하면 `아이디어`로부터 발생된 (L, R) 이 모두 사라지게 되기 때문에 `아이디`가 명사로 판별될 수 있습니다.

`soynlp.noun.LRNounExtractor_v2` 는 이러한 기능을 포함하여 그 외의 후처리 기능들을 포함하고 있습니다.

In [7]:
for word in ['아이디', '아이디어']:
    print('score({}) = {:.3}'.format(word, predict(word)))

score(아이디) = -0.704
score(아이디어) = 0.859


In [10]:
get_r('아이디', 10)

[('어', 109),
 ('어를', 103),
 ('어가', 50),
 ('', 41),
 ('는', 27),
 ('어와', 21),
 ('를', 14),
 ('어는', 12),
 ('어의', 10),
 ('어로', 9)]

In [12]:
for r, count in get_r('아이디', 30):
    if r and r[0] != '어':
        print('{} ({})'.format(r, count))

는 (27)
를 (14)
와 (9)
가 (6)
스홀딩스 (3)
로 (2)
만 (1)
병원 (1)
